# Classification on Custom Sensor Dataset

This notebook demonstrates how to:
- Load data from custom files: "dataset.txt", "features", and "targets"
- Perform exploratory data analysis and preprocessing
- Scale features and split data for training/testing
 - Train and evaluate four classifiers: SVM, Logistic Regression, Random Forest, and kNN

**Files Expected:**
 - dataset.txt: Contains the sensor readings, whitespace-delimited.
 - features: Contains feature indices and names (e.g., "1 tBodyAcc-mean()-X").
 - targets: Contains one target label per row.

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

import warnings
warnings.filterwarnings("ignore")